# 1. Ambil Data Dari Gdelt, Ambil yang Nopember Saja

In [18]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if (str(x[0:6]) == '201811')]
print(file_list)
#print("Done!")

['20181103.export.CSV.zip', '20181102.export.CSV.zip', '20181101.export.CSV.zip']


# 2. Masukkan Data ke Dalam Format TSV untuk Data Dari Negara Indonesia

In [ ]:
infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'D:/GDELT/'
dir_country = 'country'
fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(compressed_file),
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print('Downloading....')
        urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
    
    # extract the contents of the compressed file to a temporary directory    
    print('extracting,')
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print('parsing,')
    
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+dir_country+'/'+fips_country_code+'%04i.tsv'%outfilecounter
        
        try:
            # Create target Directory
            os.mkdir(local_path+dir_country)
            print("Directory " , dir_country ,  " Created ") 
        except FileExistsError:
            print("Directory " , dir_country ,  " already exists")
        
        # open the infile and outfile
        print(infile_name)
        with open(infile_name,mode='r') as infile, open(outfile_name,mode='w') as outfile:
            try:
                for line in infile:
                    # extract lines with our interest country code
                    if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                        outfile.write(line)
            except:
                print("")
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print('done')

20181103.export.CSV.zip
Downloading....
extracting,
parsing,
Directory  country  already exists
D:/GDELT/tmp\20181103.export.CSV

done
20181102.export.CSV.zip
Downloading....


# 3. Masukkan Data Dari TSV ke Database

In [ ]:
import glob
import pymysql
import pandas as pd

host = '127.0.0.1'
port = "3306"
user = 'root'
password = ''
database = 'news'

conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

def run(sql):
    df = pd.read_sql_query(sql,conn)
    return df
x = conn.cursor()

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path+'country/'+fips_country_code+'*')
DFlist = []
print("We are working on it, please take your coffee...")  
for active_file in files:
    with open(active_file,mode='r') as xfile:
        for line in xfile:
            kolom = line.split("\t")
            kolomID = kolom[0]
            kolomURL = kolom[57]
       
            sql = "INSERT INTO gdelt_content (GLOBALEVENTID,URL) SELECT * FROM (SELECT '"+kolomID+"','"+kolomURL+"') AS tmp WHERE NOT EXISTS (SELECT GLOBALEVENTID FROM gdelt_content WHERE GLOBALEVENTID = '"+kolomID+"') LIMIT 1"
            #print(sql)
            x.execute(sql)
            conn.commit()
print("Done! All have finished")
    
    
    



# 4. Update Title dan Content data di Database 

In [ ]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}
print('Take your coffee again! We will do this job for you...')
with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL)"
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r["URL"] = r["URL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

print("Done!")

SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL)
{'GLOBALEVENTID': 799369621, 'URL': 'https://www.upstreamonline.com/hardcopy/1620643/china-facing-potential-winter-gas-supply-crunch\n', 'title': None, 'content': None}
https://www.upstreamonline.com/hardcopy/1620643/china-facing-potential-winter-gas-supply-crunch

{'GLOBALEVENTID': 799369769, 'URL': 'https://www.adelaidenow.com.au/news/lion-air-crash-pilot-on-previous-flight-made-distress-call-before-continuing-to-fly/news-story/7fa1bd3b49f4dbe76444f27cc52bca41?nk=f4e3e3ca16e05848f87200a6e6df486d-1541116081\n', 'title': None, 'content': None}
https://www.adelaidenow.com.au/news/lion-air-crash-pilot-on-previous-flight-made-distress-call-before-continuing-to-fly/news-story/7fa1bd3b49f4dbe76444f27cc52bca41?nk=f4e3e3ca16e05848f87200a6e6df486d-1541116081

{'GLOBALEVENTID': 799369770, 'URL': 'http://www.jordantimes.com/news/world/pilot-radioed-alert-doomed-indonesian-jets-previous-flight\n', 'title': None, 'content': None}
http://www.jorda

{'GLOBALEVENTID': 799372283, 'URL': 'https://www.adelaidenow.com.au/news/lion-air-crash-pilot-on-previous-flight-made-distress-call-before-continuing-to-fly/news-story/7fa1bd3b49f4dbe76444f27cc52bca41?nk=f4e3e3ca16e05848f87200a6e6df486d-1541116081\n', 'title': None, 'content': None}
https://www.adelaidenow.com.au/news/lion-air-crash-pilot-on-previous-flight-made-distress-call-before-continuing-to-fly/news-story/7fa1bd3b49f4dbe76444f27cc52bca41?nk=f4e3e3ca16e05848f87200a6e6df486d-1541116081

{'GLOBALEVENTID': 799372402, 'URL': 'https://www.adelaidenow.com.au/news/lion-air-crash-pilot-on-previous-flight-made-distress-call-before-continuing-to-fly/news-story/7fa1bd3b49f4dbe76444f27cc52bca41?nk=f4e3e3ca16e05848f87200a6e6df486d-1541116081\n', 'title': None, 'content': None}
https://www.adelaidenow.com.au/news/lion-air-crash-pilot-on-previous-flight-made-distress-call-before-continuing-to-fly/news-story/7fa1bd3b49f4dbe76444f27cc52bca41?nk=f4e3e3ca16e05848f87200a6e6df486d-1541116081

{'GLOBAL

{'GLOBALEVENTID': 799380937, 'URL': 'https://www.news.com.au/national/politics/malcolm-turnbull-is-ready-to-give-his-version-of-events-and-there-is-one-question-we-all-want-to-know/news-story/59ecd3823f4935e963483d4ea16f6bcd\n', 'title': None, 'content': None}
https://www.news.com.au/national/politics/malcolm-turnbull-is-ready-to-give-his-version-of-events-and-there-is-one-question-we-all-want-to-know/news-story/59ecd3823f4935e963483d4ea16f6bcd

{'GLOBALEVENTID': 799381130, 'URL': 'https://www.sbs.com.au/news/lion-air-pilot-wanted-to-turn-ill-fated-plane-back-on-earlier-flight\n', 'title': None, 'content': None}
https://www.sbs.com.au/news/lion-air-pilot-wanted-to-turn-ill-fated-plane-back-on-earlier-flight

{'GLOBALEVENTID': 799381131, 'URL': 'https://www.sbs.com.au/news/lion-air-pilot-wanted-to-turn-ill-fated-plane-back-on-earlier-flight\n', 'title': None, 'content': None}
https://www.sbs.com.au/news/lion-air-pilot-wanted-to-turn-ill-fated-plane-back-on-earlier-flight

{'GLOBALEVENTI

{'GLOBALEVENTID': 799385495, 'URL': 'https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/\n', 'title': None, 'content': None}
https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/

{'GLOBALEVENTID': 799385496, 'URL': 'https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/\n', 'title': None, 'content': None}
https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/

{'GLOBALEVENTID': 799385497, 'URL': 'https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/\n', 'title': None, 'content': None}
https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/

{'GLOBALEVENTID': 799385498, 'URL': 'https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/\n', 'title': None, 'content': None}
https://www.thestar.com.my/news/regional/2018/11/02/tin-islands-hit-hard-by-crash/

{'GLOBALEVENTID': 799385500, 'URL': 'htt

{'GLOBALEVENTID': 799389623, 'URL': 'https://www.thenews.com.pk/print/388490-as-imran-khan-visits-china\n', 'title': None, 'content': None}
https://www.thenews.com.pk/print/388490-as-imran-khan-visits-china

{'GLOBALEVENTID': 799389624, 'URL': 'https://www.couriermail.com.au/news/world/an-indonesian-maid-has-been-executed-in-saudi-arabia-for-killing-her-employer-during-a-rape-attack/news-story/8d5ee3f6d75ff93223d90e34df4f99d1?nk=b1767b2c25535a33e2ded501206cc2d1-1541119586\n', 'title': None, 'content': None}
https://www.couriermail.com.au/news/world/an-indonesian-maid-has-been-executed-in-saudi-arabia-for-killing-her-employer-during-a-rape-attack/news-story/8d5ee3f6d75ff93223d90e34df4f99d1?nk=b1767b2c25535a33e2ded501206cc2d1-1541119586

{'GLOBALEVENTID': 799389625, 'URL': 'https://www.couriermail.com.au/news/world/an-indonesian-maid-has-been-executed-in-saudi-arabia-for-killing-her-employer-during-a-rape-attack/news-story/8d5ee3f6d75ff93223d90e34df4f99d1?nk=b1767b2c25535a33e2ded501206cc

...

CRITICAL:newspaper.article:Other error with setting top image using the Reddit algorithm. Possible error with PIL, Invalid header value b'http://www.ghanaiantimes.com.gh/indonesia-plane-crash-search-detected-black-box/\n'


{'GLOBALEVENTID': 799479175, 'URL': 'https://www.thetimes.co.uk/article/new-boeing-737-crashes-into-sea-off-jakarta-with-189-on-board-2f76swbq2\n', 'title': None, 'content': None}
https://www.thetimes.co.uk/article/new-boeing-737-crashes-into-sea-off-jakarta-with-189-on-board-2f76swbq2

{'GLOBALEVENTID': 799479196, 'URL': 'https://www.business-standard.com/article/pti-stories/thousands-protest-burning-of-islamic-flag-in-indonesia-118110200738_1.html\n', 'title': None, 'content': None}
https://www.business-standard.com/article/pti-stories/thousands-protest-burning-of-islamic-flag-in-indonesia-118110200738_1.html

{'GLOBALEVENTID': 799480952, 'URL': 'https://www.ctpost.com/news/world/article/Thousands-protest-burning-of-Islamic-flag-in-13356945.php\n', 'title': None, 'content': None}
https://www.ctpost.com/news/world/article/Thousands-protest-burning-of-Islamic-flag-in-13356945.php

{'GLOBALEVENTID': 799481195, 'URL': 'https://punchng.com/search-teams-recover-seats-wheels-from-indonesia-

{'GLOBALEVENTID': 799486787, 'URL': 'https://www.malaysiakini.com/news/450142\n', 'title': None, 'content': None}
https://www.malaysiakini.com/news/450142

{'GLOBALEVENTID': 799486826, 'URL': 'https://www.malaysiakini.com/news/450142\n', 'title': None, 'content': None}
https://www.malaysiakini.com/news/450142

{'GLOBALEVENTID': 799487067, 'URL': 'http://interaksyon.com/politics-issues/2018/11/02/137296/ofw-jennifer-dalquez-home-after-release-from-uae-death-row/\n', 'title': None, 'content': None}
http://interaksyon.com/politics-issues/2018/11/02/137296/ofw-jennifer-dalquez-home-after-release-from-uae-death-row/

{'GLOBALEVENTID': 799487068, 'URL': 'http://interaksyon.com/politics-issues/2018/11/02/137296/ofw-jennifer-dalquez-home-after-release-from-uae-death-row/\n', 'title': None, 'content': None}
http://interaksyon.com/politics-issues/2018/11/02/137296/ofw-jennifer-dalquez-home-after-release-from-uae-death-row/

{'GLOBALEVENTID': 799487256, 'URL': 'https://www.foxnews.com/world/thous

{'GLOBALEVENTID': 799492907, 'URL': 'https://www.emirates247.com/news/world/search-teams-recover-seats-wheels-from-indonesia-jet-crash-site-2018-11-02-1.674893\n', 'title': None, 'content': None}
https://www.emirates247.com/news/world/search-teams-recover-seats-wheels-from-indonesia-jet-crash-site-2018-11-02-1.674893

{'GLOBALEVENTID': 799492949, 'URL': 'http://www.xinhuanet.com/english/2018-11/02/c_137577097.htm\n', 'title': None, 'content': None}
http://www.xinhuanet.com/english/2018-11/02/c_137577097.htm

{'GLOBALEVENTID': 799492957, 'URL': 'http://en.tempo.co/read/news/2018/11/02/055923111/KNKT-to-Take-3-Steps-to-Analyze-Lion-Air-JT-610-Black-Box\n', 'title': None, 'content': None}
http://en.tempo.co/read/news/2018/11/02/055923111/KNKT-to-Take-3-Steps-to-Analyze-Lion-Air-JT-610-Black-Box

{'GLOBALEVENTID': 799492958, 'URL': 'http://en.tempo.co/read/news/2018/11/02/055923111/KNKT-to-Take-3-Steps-to-Analyze-Lion-Air-JT-610-Black-Box\n', 'title': None, 'content': None}
http://en.tempo

{'GLOBALEVENTID': 799495542, 'URL': 'https://www.dailymail.co.uk/news/article-6345661/Lion-Air-plane-totally-destroyed-pieces-sea-bed-crashing-tremendous-speed.html\n', 'title': None, 'content': None}
https://www.dailymail.co.uk/news/article-6345661/Lion-Air-plane-totally-destroyed-pieces-sea-bed-crashing-tremendous-speed.html

{'GLOBALEVENTID': 799496015, 'URL': 'https://abcnews.go.com/International/wireStory/details-highlight-lion-air-jets-problems-crash-58917444\n', 'title': None, 'content': None}
https://abcnews.go.com/International/wireStory/details-highlight-lion-air-jets-problems-crash-58917444

{'GLOBALEVENTID': 799496049, 'URL': 'https://www.dailymail.co.uk/news/article-6345661/Lion-Air-plane-totally-destroyed-pieces-sea-bed-crashing-tremendous-speed.html\n', 'title': None, 'content': None}
https://www.dailymail.co.uk/news/article-6345661/Lion-Air-plane-totally-destroyed-pieces-sea-bed-crashing-tremendous-speed.html

{'GLOBALEVENTID': 799496050, 'URL': 'https://www.dailymail.c